# Imports

In [2]:
import json
import re
from pprint import pprint
import pandas as pd
import torch
from datasets import Dataset
from peft import LoraConfig, PeftModel
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer

2023-12-09 13:22:22.119924: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 13:22:22.119954: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 13:22:22.121019: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 13:22:22.126533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-09 13:22:22.806008: W tensorflow/compiler/tf2

In [3]:
import torch
torch.cuda.empty_cache()

# Load Dataset

In [4]:
dataset = pd.read_csv("FinalInstructImproved.csv")

In [5]:
dataset.head()

,Unnamed: 0,Notebook,Position,ExtractedPrompt,ImprovedPrompt,Code
0,0,1481,1,# Generating text for NLP using SimpleRNN with...,Write Python code to import the following libr...,# This Python 3 environment comes with many he...
1,1,1481,2,## 2. Import data and preprocessing.\n\nWe rea...,"Write Python code to read a text file named ""a...","fin = open(""../input/alice_in_wonderland.txt"",..."
2,2,1481,3,The next step is to create the input and label...,"Write Python code to create two lists, input_c...",SEQLEN = 10\nSTEP = 1\ninput_chars = []\nlabel...
3,3,1481,4,The next step is to vectorize these input and ...,"Write Python code to create two arrays, X and ...","X = np.zeros((len(input_chars), SEQLEN, nb_cha..."
4,4,1481,5,## 3. Model Building.\n\n* We define the RNN's...,Write Python code to define a Sequential model...,HIDDEN_SIZE = 128\nBATCH_SIZE = 128\nNUM_ITERA...


In [6]:
SYSTEM_PROMPT = """
You are a Data Science Expert. Write the python code outlined in the instruction below.
""".strip()

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
train_unclean, test_unclean = train_test_split(dataset, test_size=0.2, random_state=42)

In [9]:
train_unclean, val_unclean = train_test_split(train_unclean, test_size=0.1, random_state=42)

In [10]:
print(f"train shape: {train_unclean.shape}")
print(f"val shape: {val_unclean.shape}")
print(f"test shape: {test_unclean.shape}")

train shape: (6696, 6)
val shape: (745, 6)
test shape: (1861, 6)


## Add column to dataframe with formatted instruction

In [11]:
def process_dataframe(df, test=False):
    data = df.copy()
    def format_instruction(row, test):
        if test: 
            return f"""
        <s>[INST] <<SYS>>\nYou are an expert in Data Science. Below is an instruction that describes a task. Write Python code that appropriately completes the request. Please wrap your code using ```.\n<</SYS>>\n\n{row['ImprovedPrompt']}[/INST]
        """.strip()
        return f"""
        <s>[INST] <<SYS>>\nYou are an expert in Data Science. Below is an instruction that describes a task. Write Python code that appropriately completes the request. Please wrap your code using ```.\n<</SYS>>\n\n{row['ImprovedPrompt']}[/INST] ```\n{row['Code']}\n``` </s>
        """.strip()
    data['text'] = data.apply(lambda x: format_instruction(x, test), axis=1)
    if test:
        data['baseline_non_instruct'] = data.apply(lambda x: format_instruction(x, test), axis=1)
    return data

In [12]:
train_df = process_dataframe(train_unclean)
val_df = process_dataframe(val_unclean)
test_df = process_dataframe(test_unclean, test=True)
test_df.to_csv("../experiments/test_formatted.csv")
train = Dataset.from_pandas(train_df)
val = Dataset.from_pandas(val_df)
test = Dataset.from_pandas(test_df)

In [13]:
test = pd.read_csv("../experiments/test_formatted.csv")
test.head()

,Unnamed: 0.1,Unnamed: 0,Notebook,Position,ExtractedPrompt,ImprovedPrompt,Code,text,baseline_non_instruct
0,6032,6145,1771,5,"## Section 1.3 Resize, Flip and Rotate ### Sub...",Write Python code to read an image file named ...,"lena_rgb = cv2.imread(lena, cv2.IMREAD_UNCHANG...",<s>[INST] <<SYS>>\nYou are an expert in Data S...,<s>[INST] <<SYS>>\nYou are an expert in Data S...
1,1112,1138,2087,5,"Since we are all here to learn, let's explore ...",Write Python code to extract the advice from a...,advice = quote[quote.new_question == 562]\npri...,<s>[INST] <<SYS>>\nYou are an expert in Data S...,<s>[INST] <<SYS>>\nYou are an expert in Data S...
2,7990,8133,3014,29,- From the above visualization we can see that...,Write Python code to import the Seaborn librar...,"ax = sns.countplot(x=df.Alcoholism, hue=df.NoS...",<s>[INST] <<SYS>>\nYou are an expert in Data S...,<s>[INST] <<SYS>>\nYou are an expert in Data S...
3,8271,8418,3021,5,### Numerical Variables,Write Python code to create a figure with a si...,"plt.figure(figsize=(14, len(num_cols) * 3))\n\...",<s>[INST] <<SYS>>\nYou are an expert in Data S...,<s>[INST] <<SYS>>\nYou are an expert in Data S...
4,6352,6469,2548,18,# Some Resources:* https://keras.io/\n* https:...,Write Python code to import the IPython.displa...,"from IPython.display import display, HTML\n\nc...",<s>[INST] <<SYS>>\nYou are an expert in Data S...,<s>[INST] <<SYS>>\nYou are an expert in Data S...


# Instantiate Model

In [18]:
MODEL_NAME = "codellama/CodeLlama-7b-Instruct-hf"
OUTPUT_DIR = "turing-7b-2epoch-12-08-2023"

In [19]:
def create_model_and_tokenizer():
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    )
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_NAME,
        use_safetensors=True,
        quantization_config=bnb_config,
        trust_remote_code=True,
        device_map="auto",
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "right"

    return model, tokenizer

In [20]:
model, tokenizer = create_model_and_tokenizer()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [21]:
model.config.quantization_config.to_dict()

{'quant_method': <QuantizationMethod.BITS_AND_BYTES: 'bitsandbytes'>,
 'load_in_8bit': False,
 'load_in_4bit': True,
 'llm_int8_threshold': 6.0,
 'llm_int8_skip_modules': None,
 'llm_int8_enable_fp32_cpu_offload': False,
 'llm_int8_has_fp16_weight': False,
 'bnb_4bit_quant_type': 'nf4',
 'bnb_4bit_use_double_quant': False,
 'bnb_4bit_compute_dtype': 'float16'}

## LORA config

In [22]:
lora_alpha = 16
lora_dropout = 0.05
lora_r = 8

peft_config = LoraConfig(
    lora_alpha=lora_alpha, 
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

### Training parameters

In [24]:
training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=3,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="constant",
    seed=42,
)   

In [25]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train,
    eval_dataset=val,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=1024,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/6696 [00:00<?, ? examples/s]

Map:   0%|          | 0/745 [00:00<?, ? examples/s]

### Training

In [26]:
trainer.train()

You're using a CodeLlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
251,0.528000,0.596497
502,0.531500,0.557380
753,0.558200,0.544338
1004,0.562500,0.539134


TrainOutput(global_step=1254, training_loss=0.5681097306655735, metrics={'train_runtime': 6184.9766, 'train_samples_per_second': 3.248, 'train_steps_per_second': 0.203, 'total_flos': 3.359670957246382e+17, 'train_loss': 0.5681097306655735, 'epoch': 3.0})

In [27]:
trainer.save_model()

In [28]:
from peft import AutoPeftModelForCausalLM

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    OUTPUT_DIR,
    low_cpu_mem_usage=True,
)

merged_model = trained_model.merge_and_unload()
merged_model.save_pretrained("merged_model_train_cole", safe_serialization=True)
tokenizer.save_pretrained("merged_model_train_cole")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

('merged_model_train_cole/tokenizer_config.json',
 'merged_model_train_cole/special_tokens_map.json',
 'merged_model_train_cole/tokenizer.json')

## Run on one example

Okay I found the issue. We have to RELOAD the model WITHOUT peft or any funny business, before loading it with PEFT. 

In [ ]:
model, tokenizer = create_model_and_tokenizer()

In [ ]:
trained_model = PeftModel.from_pretrained(model, OUTPUT_DIR)

In [ ]:
test_text = test['text'][200]

In [ ]:
tokenized_test = tokenizer(test_text, return_tensors='pt').to("cuda")

In [ ]:
%%time
with torch.inference_mode():
    toks = trained_model.generate(**tokenized_test, max_new_tokens=1024)[0]

In [ ]:
res = tokenizer.decode(toks, skip_special_tokens=True)

In [ ]:
print(res)

## 